## Notebook Version

1. Creator : Diego Mendez
2. Version  : 1. Creacion de Codigo Makro-Extraccion Oracle

In [1]:
pip install selenium beautifulsoup4 pandas webdriver_manager openpyxl pandas-gbq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Libraries

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import os
import re
import glob
import pandas as pd
from google.oauth2 import service_account

#### Clase Makro Automatización

In [3]:

class MakroAutomation:
    """Clase para automatizar el proceso de obtención de reportes desde el portal B2B de Makro."""
    
    def __init__(self, credentials_file="credentials.txt"):
        """Inicializa la automatización de Makro.
        
        Args:
            credentials_file (str): Ruta al archivo que contiene las credenciales.
                                   El archivo debe tener el formato:
                                   username=usuario@ejemplo.com
                                   password=contraseña
        """
        self.credentials_file = credentials_file
        self.username, self.password = self._load_credentials()
        self.driver = None
        self.main_window = None
        
        # Ejecutar la secuencia de automatización
        self.run_automation()
        
    def _load_credentials(self):
        """Carga las credenciales desde un archivo de texto.
        
        Returns:
            tuple: (username, password) leídos del archivo.
            
        Raises:
            FileNotFoundError: Si el archivo de credenciales no existe.
            ValueError: Si el formato del archivo es incorrecto.
        """
        try:
            username = None
            password = None
            
            with open(self.credentials_file, 'r') as file:
                for line in file:
                    line = line.strip()
                    if line.startswith('username='):
                        username = line.split('=', 1)[1]
                    elif line.startswith('password='):
                        password = line.split('=', 1)[1]
            
            if not username or not password:
                raise ValueError(f"Formato incorrecto en el archivo {self.credentials_file}. " 
                                "Debe contener líneas con 'username=' y 'password='")
                
            print(f"-->Credenciales cargadas exitosamente desde {self.credentials_file}")
            return username, password
            
        except FileNotFoundError:
            print(f"ERROR: Archivo de credenciales '{self.credentials_file}' no encontrado.")
            print("Usando credenciales predeterminadas para propósitos de desarrollo.")
            # Valores predeterminados como fallback (solo para desarrollo)
            return "grandessuperficies@donchicharron.com.co", "Pilar2025"
        except Exception as e:
            print(f"ERROR al cargar credenciales: {e}")
            print("Usando credenciales predeterminadas para propósitos de desarrollo.")
            # Valores predeterminados como fallback (solo para desarrollo)
            return "grandessuperficies@donchicharron.com.co", "Pilar2025"
        
    def initialize_driver(self):
        """Initialize and configure the Edge webdriver."""
        self.driver = webdriver.Chrome()  # O Firefox, Edge, etc.
        self.driver.implicitly_wait(10)
        return self.driver
    
    def login(self):
        """Realiza el inicio de sesión en el portal B2B de Makro."""
        try:
            self.driver.get("https://b2b.makro.com/")
            
            username_field = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "usernameField"))
            )
            
            username_field.click()
            username_field.clear()
            print("-->Inicio de sesion")
            username_field.send_keys(self.username)
            
            pass_field = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "passwordField"))
            )
            pass_field.click()
            pass_field.send_keys(self.password)
            
            login_button = self.driver.find_element(By.XPATH, "//button[@message='FND_SSO_LOGIN']")
            login_button.click()
            
            # Esperar a que la página cargue después del login
            time.sleep(3)
            return True
            
        except TimeoutException:
            print("Tiempo de espera agotado durante el inicio de sesión.")
            return False
        except Exception as e:
            print(f"Error durante el inicio de sesión: {e}")
            return False
    
    def navigate_to_isupplier(self):
        """Navega a la sección ISUPPLIER VENDOR COMMERCIAL."""
        try:
            isupplier_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//div[@class='textdivresp' and text()='ISUPPLIER VENDOR COMMERCIAL']"))
            )
            print("-->Botón 'ISUPPLIER VENDOR COMMERCIAL' encontrado por texto exacto")
            isupplier_button.click()
            print("-->Botón 'ISUPPLIER VENDOR COMMERCIAL' clickeado1")
            
            # Esperar a que la página cargue
            time.sleep(3)
            return True
            
        except TimeoutException:
            print("Tiempo de espera agotado al buscar ISUPPLIER VENDOR COMMERCIAL.")
            return False
        except Exception as e:
            print(f"Error al navegar a ISUPPLIER: {e}")
            return False
    
    def navigate_to_commercial(self):
        """Navega a la sección Commercial."""
        try:
            commercial_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.ID, "MAKRO_POS_COMMERCIAL"))
            )
            print("-->Botón 'Commercial' encontrado por ID")
            commercial_button.click()
            
            # Esperar a que la página cargue
            time.sleep(5)
            return True
            
        except TimeoutException:
            print("No se pudo encontrar el botón 'Commercial'")
            return False
        except Exception as e:
            print(f"Error al navegar a Commercial: {e}")
            return False
    
    def navigate_to_stock_po_sales(self):
        """Navega al reporte de Stock and PO and Sales."""
        try:
            stock_po_sales_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.ID, "STOCKPOSALES"))
            )
            print("-->Botón 'Stock and PO and Sales report' encontrado por ID")
            stock_po_sales_button.click()
            
            # Esperar a que la nueva ventana se abra
            time.sleep(10)
            return True
            
        except TimeoutException:
            print("No se pudo encontrar el botón 'Stock and PO and Sales report'")
            return False
        except Exception as e:
            print(f"Error al navegar a Stock PO Sales: {e}")
            return False
    
    def switch_to_report_window(self):
        """Cambia el foco a la ventana del reporte."""
        try:
            self.main_window = self.driver.current_window_handle
            
            all_windows = self.driver.window_handles
            for window in all_windows:
                if window != self.main_window:
                    self.driver.switch_to.window(window)
                    print("-->Cambiado a la nueva ventana del reporte")
                    return True
            
            print("No se encontró una nueva ventana.")
            return False
            
        except Exception as e:
            print(f"Error al cambiar a la ventana del reporte: {e}")
            return False
    
    def configure_report_parameters(self, start_date="01/11/2025", end_date="11/11/2025"):
        """Configura los parámetros del reporte.
        
        Args:
            start_date (str): Fecha de inicio en formato DD/MM/YYYY.
            end_date (str): Fecha de fin en formato DD/MM/YYYY.
        """
        try:
            # Configurar fecha de inicio
            begin_date_field = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.ID, "_paramsPM_BEGIN_DATE"))
            )
            print("-->Campo de fecha de inicio encontrado")
            
            begin_date_field.clear()
            begin_date_field.send_keys(start_date)
            print(f"-->Fecha de inicio ingresada: {start_date}")
            
            # Configurar fecha de fin
            end_date_field = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.ID, "_paramsPM_END_DATE"))
            )
            end_date_field.clear()
            end_date_field.send_keys(end_date)
            print(f"-->Fecha final ingresada: {end_date}")
            
            # Configurar "Totalizar tiendas"
            self._select_dropdown_option(
                dropdown_id="xdo:xdo:_paramsPM_SUM_LOCATIONS_div_input",
                option_xpath="//li[contains(@id, '_paramsPM_SUM_LOCATIONS_div_li') and .//div[text()='No']]",
                dropdown_name="Totalizar tiendas"
            )
            
            # Configurar "Mostrar total empresa"
            self._select_dropdown_option(
                dropdown_id="xdo:xdo:_paramsPM_SHOW_TOTAL_COMPANY_div_input",
                option_xpath="//li[contains(@id, '_paramsPM_SHOW_TOTAL_COMPANY_div_li') and .//div[text()='No']]",
                dropdown_name="Mostrar total empresa"
            )
            
            # Aplicar cambios
            apply_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.ID, "reportViewApply"))
            )
            apply_button.click()
            print("-->Botón 'Aplicar' presionado")
            
            # Esperar a que el reporte se genere
            return True
            
        except TimeoutException as e:
            print(f"Error al interactuar con el formulario de fechas: {e}")
            return False
        except Exception as e:
            print(f"Error inesperado en configuración de parámetros: {e}")
            return False
    
    def _select_dropdown_option(self, dropdown_id, option_xpath, dropdown_name):
        """Selecciona una opción de un dropdown.
        
        Args:
            dropdown_id (str): ID del elemento dropdown.
            option_xpath (str): XPath de la opción a seleccionar.
            dropdown_name (str): Nombre descriptivo del dropdown (para logs).
        """
        time.sleep(5)
        dropdown = WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.ID, dropdown_id))
        )
        dropdown.click()
        print(f"-->Dropdown '{dropdown_name}' abierto")
        
        # Pequeña pausa para que se despliegue el dropdown
        time.sleep(1)
        
        no_option = WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, option_xpath))
        )
        no_option.click()
        print(f"-->Opción 'No' seleccionada para '{dropdown_name}'")
    
    def download_excel_report(self):
        """Descarga el reporte en formato Excel."""
        try:
            print("-->Esperando que el reporte se genere")
            
            # Hacer clic en "Ver Informe"
            view_report_link = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.ID, "xdo:viewFormatLink"))
            )
            print("-->Esperando el enlace 'Ver Informe'")
            time.sleep(2)
            view_report_link.click()
            print("-->Enlace 'Ver Informe' presionado")
            time.sleep(120)
            
            # Intentar seleccionar la opción Excel por texto
            try:
                excel_option = WebDriverWait(self.driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'floatMenu') and contains(@style, 'display: block')]//div[contains(@class, 'itemTxt') and text()='Excel (*.xlsx)']"))
                )
                
                excel_option.click()
                print("-->Opción 'Excel (*.xlsx)' seleccionada mediante texto")
            except TimeoutException:
                # Método alternativo si el primero falla
                visible_menu = WebDriverWait(self.driver, 5).until(
                    EC.visibility_of_element_located((By.XPATH, "//div[contains(@class, 'floatMenu') and contains(@style, 'display: block')]"))
                )
                excel_menu_item = visible_menu.find_element(By.XPATH, ".//li[@fmid='3' or @fmid='102']")
                excel_link = excel_menu_item.find_element(By.TAG_NAME, "a")
                print("-->Esperando el enlace 'Excel'")
                
                excel_link.click()
                print("Opción 'Excel (*.xlsx)' seleccionada mediante fmid")
            print("Esperando que el archivo se descargue")
            
            print("-->Esperando que el archivo empiece a cargar en pantalla")   
            print("-->Descarga del archivo Excel iniciada")
            return True
            
        except Exception as e:
            print(f"Error al seleccionar la opción de descarga: {e}")
            return False
    
    def run_automation(self):
        """Ejecuta la secuencia completa de automatización."""
        try:
            self.initialize_driver()
    
            if not self.login():
                raise Exception("Fallo en el inicio de sesión")
            
            if not self.navigate_to_isupplier():
                raise Exception("Fallo en la navegación a ISUPPLIER")
            
            if not self.navigate_to_commercial():
                raise Exception("Fallo en la navegación a Commercial")
            
            if not self.navigate_to_stock_po_sales():
                raise Exception("Fallo en la navegación a Stock PO Sales")
            
            if not self.switch_to_report_window():
                raise Exception("Fallo al cambiar a la ventana del reporte")
            
            # Configurar parámetros del reporte
            if not self.configure_report_parameters():
                raise Exception("Fallo en la configuración de parámetros")
            
            # Descargar reporte en Excel
            if not self.download_excel_report():
                raise Exception("Fallo en la descarga del reporte")
            
            print("-->Automatización completada exitosamente")
            
        except Exception as e:
            print(f"Error durante la automatización: {e}")
        finally:
            # No cerramos el driver automáticamente para mantener el comportamiento original
            pass
    
   

#### Inicializacion de la funcion

In [4]:
if __name__ == "__main__":

    automation = MakroAutomation()

-->Credenciales cargadas exitosamente desde credentials.txt
-->Inicio de sesion
-->Botón 'ISUPPLIER VENDOR COMMERCIAL' encontrado por texto exacto
-->Botón 'ISUPPLIER VENDOR COMMERCIAL' clickeado1
-->Botón 'Commercial' encontrado por ID
Error al navegar a Stock PO Sales: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=142.0.7444.134)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6e62c7a05
	0x7ff6e62c7a60
	0x7ff6e60416ad
	0x7ff6e602d1c5
	0x7ff6e6052a5a
	0x7ff6e60ca306
	0x7ff6e60eb222
	0x7ff6e608b068
	0x7ff6e608be93
	0x7ff6e65829a0
	0x7ff6e657ce20
	0x7ff6e659cc15
	0x7ff6e62e309e
	0x7ff6e62ead8f
	0x7ff6e62d0be4
	0x7ff6e62d0d9f
	0x7ff6e62b67f8
	0x7fff04b17374
	0x7fff06abcc91

Error durante la automatización: Fallo en la navegación a Stock PO Sales


In [5]:
"""
Módulo para procesamiento de datos de Excel y carga a BigQuery.

Este módulo proporciona clases y funcionalidades para procesar archivos Excel,
transformar los datos y cargarlos en Google BigQuery.
"""



class ExcelProcessor:
    """
    Clase para procesar archivos Excel y consolidar sus datos.
    
    Esta clase proporciona métodos para buscar, leer y procesar archivos
    Excel en un directorio, combinando sus datos en un único DataFrame.
    """
    
    def __init__(self, directorio="../../", patron="*PO*", header=16):
        """
        Inicializa el procesador de archivos Excel.
        
        Args:
            directorio (str): Ruta al directorio donde buscar archivos.
            patron (str): Patrón para filtrar los archivos (ej: "*PO*").
            header (int): Número de fila que contiene los encabezados (0-based).
        """
        self.directorio = directorio
        self.patron = patron
        self.header = header
        self.archivos = []
        self.dataframes = []
        
    def buscar_archivos(self):
        """
        Busca archivos que coincidan con el patrón especificado.
        
        Returns:
            list: Lista de rutas a los archivos encontrados.
        """
        self.archivos = glob.glob(os.path.join(self.directorio, self.patron))
        return self.archivos
    
    def procesar_archivos(self, mostrar_stats=True):
        """
        Procesa todos los archivos Excel encontrados.
        
        Args:
            mostrar_stats (bool): Si es True, muestra estadísticas del proceso.
            
        Returns:
            pandas.DataFrame: DataFrame con todos los datos concatenados.
        """
        if not self.archivos:
            self.buscar_archivos()
            
        self.dataframes = []
        
        total_archivos = len(self.archivos)
        if mostrar_stats:
            print(f"Se encontraron {total_archivos} archivos con el patrón '{self.patron}'")
        
        for i, archivo in enumerate(self.archivos, 1):
            if mostrar_stats:
                print(f"[{i}/{total_archivos}] Procesando archivo: {archivo}")
            
            try:
                engine = self._determinar_engine(archivo)
                if not engine and mostrar_stats:
                    print(f"  - Archivo {archivo} no es un archivo Excel reconocido - omitido")
                    continue
                
                dict_dfs = pd.read_excel(archivo, sheet_name=None, header=self.header, engine=engine)
                
                self._procesar_hojas(dict_dfs, archivo, mostrar_stats)
                
            except Exception as e:
                if mostrar_stats:
                    print(f"  - Error al procesar el archivo {archivo}: {str(e)}")
        
        return self.obtener_dataframe_consolidado(mostrar_stats)
    
    def _determinar_engine(self, archivo):
        """
        Determina el motor adecuado para leer el archivo Excel.
        
        Args:
            archivo (str): Ruta al archivo Excel.
            
        Returns:
            str: Motor a utilizar ('openpyxl' o 'xlrd') o None si no es un archivo Excel reconocido.
        """
        if archivo.endswith('.xlsx'):
            return 'openpyxl'
        elif archivo.endswith('.xls'):
            return 'xlrd'
        return None
    
    def _procesar_hojas(self, dict_dfs, archivo, mostrar_stats):
        """
        Procesa cada hoja de un archivo Excel.
        
        Args:
            dict_dfs (dict): Diccionario con los DataFrames de cada hoja.
            archivo (str): Ruta al archivo Excel.
            mostrar_stats (bool): Si es True, muestra estadísticas del proceso.
        """
        for nombre_hoja, hoja_df in dict_dfs.items():
            if not hoja_df.empty:
                hoja_df['archivo_origen'] = os.path.basename(archivo)
                hoja_df['hoja_origen'] = nombre_hoja
                self.dataframes.append(hoja_df)
                if mostrar_stats:
                    print(f"  - Procesada hoja '{nombre_hoja}' con {len(hoja_df)} filas")
            elif mostrar_stats:
                print(f"  - Hoja '{nombre_hoja}' está vacía - omitida")
    
    def obtener_dataframe_consolidado(self, mostrar_stats=True):
        """
        Combina todos los DataFrames procesados en uno solo.
        
        Args:
            mostrar_stats (bool): Si es True, muestra estadísticas del DataFrame final.
            
        Returns:
            pandas.DataFrame: DataFrame con todos los datos concatenados o None si no hay datos.
        """
        if not self.dataframes:
            if mostrar_stats:
                print("No se encontraron datos válidos para concatenar.")
            return None
        
        df_final = pd.concat(self.dataframes, ignore_index=True)
        
        if mostrar_stats:
            print(f"\nDataFrame final: {len(df_final)} filas, {len(df_final.columns)} columnas")
            print("\nDistribución de filas por archivo:")
            print(df_final['archivo_origen'].value_counts())
        
        return df_final
    
    def guardar_excel(self, df, nombre_archivo="datos_concatenados.xlsx", mostrar_stats=True):
        """
        Guarda el DataFrame consolidado en un archivo Excel.
        
        Args:
            df (pandas.DataFrame): DataFrame a guardar.
            nombre_archivo (str): Nombre del archivo de salida.
            mostrar_stats (bool): Si es True, muestra mensaje de confirmación.
            
        Returns:
            bool: True si se guardó correctamente, False en caso contrario.
        """
        if df is not None:
            df.to_excel(nombre_archivo, index=False)
            if mostrar_stats:
                print(f"\nDatos guardados en: {nombre_archivo}")
            return True
        return False


class DataFrameCleaner:
    """
    Clase para limpiar y transformar DataFrames antes de subirlos a BigQuery.
    
    Esta clase proporciona métodos para normalizar nombres de columnas,
    convertir tipos de datos y aplicar transformaciones específicas.
    """
    
    @staticmethod
    def normalizar_nombres_columnas(df):
        """
        Normaliza los nombres de columnas para que sean compatibles con BigQuery.
        
        Args:
            df (pandas.DataFrame): DataFrame a limpiar.
            
        Returns:
            pandas.DataFrame: DataFrame con nombres de columnas normalizados.
        """
        df_limpio = df.copy()
        columnas_limpias = {}
        
        for col in df.columns:
            nuevo_nombre = col.strip()
            nuevo_nombre = re.sub(r'[^a-zA-Z0-9]', '_', nuevo_nombre)
            nuevo_nombre = re.sub(r'_+', '_', nuevo_nombre)
            nuevo_nombre = nuevo_nombre.strip('_')
            
            if nuevo_nombre and nuevo_nombre[0].isdigit():
                nuevo_nombre = 'col_' + nuevo_nombre
                
            if not nuevo_nombre:
                nuevo_nombre = f'columna_{df.columns.get_loc(col)}'
                
            columnas_limpias[col] = nuevo_nombre
            
        df_limpio.rename(columns=columnas_limpias, inplace=True)
        
        print("Nombres de columnas normalizados:")
        for original, nuevo in columnas_limpias.items():
            if original != nuevo:
                print(f"  • '{original}' → '{nuevo}'")
                
        return df_limpio
    
    @staticmethod
    def convertir_a_tipo_numerico(df, columnas):
        """
        Convierte columnas específicas a tipo numérico.
        
        Args:
            df (pandas.DataFrame): DataFrame a procesar.
            columnas (list): Lista de nombres de columnas a convertir.
            
        Returns:
            pandas.DataFrame: DataFrame con las columnas convertidas.
        """
        df_convertido = df.copy()
        
        for columna in columnas:
            if columna in df_convertido.columns:
                df_convertido[columna] = pd.to_numeric(df_convertido[columna], errors='coerce')
                print(f"Columna '{columna}' convertida a tipo numérico")
                
        return df_convertido
    
    @staticmethod
    def procesar_ventas(df_ventas):
        """
        Procesa el DataFrame de ventas aplicando filtros y transformaciones específicas.
        
        Args:
            df_ventas (pandas.DataFrame): DataFrame de ventas a procesar.
            
        Returns:
            pandas.DataFrame: DataFrame procesado.
        """
        # Selección de columnas relevantes
        columnas_seleccionadas = [
            "Ubicaci_n", 
            "Descripci_n", 
            "Cantidad_de_ventas", 
            "Monto_de_ventas"
        ]
        
        df_procesado = df_ventas[columnas_seleccionadas].copy()
        
        # Convertir columnas numéricas
        df_procesado["Cantidad_de_ventas"] = pd.to_numeric(df_procesado["Cantidad_de_ventas"], errors='coerce')
        df_procesado["Monto_de_ventas"] = pd.to_numeric(df_procesado["Monto_de_ventas"], errors='coerce')
        
        # Reemplazar valores NaN
        df_procesado[columnas_seleccionadas] = df_procesado[columnas_seleccionadas].replace('nan', '')
        
        # Aplicar filtros
        df_procesado = df_procesado[df_procesado["Descripci_n"] != ""]
        df_procesado = df_procesado[df_procesado["Cantidad_de_ventas"] != 0]
        df_procesado = df_procesado[~df_procesado["Descripci_n"].str.contains("CONSUMO", na=False)]
        
        return df_procesado
        
    @staticmethod
    def procesar_compras(df_compras):
        """
        Procesa el DataFrame de compras e inventario aplicando filtros y transformaciones específicas.
        
        Args:
            df_compras (pandas.DataFrame): DataFrame de compras e inventario a procesar.
            
        Returns:
            pandas.DataFrame: DataFrame procesado para análisis de inventario.
        """
        # Selección de columnas relevantes
        columnas_seleccionadas = [
            "N_mero_de_producto_Makro",
            "Descripci_n", 
            "Ubicaci_n", 
            "Cantidad_de_stock",
            "Valor_de_stock_disponible",  # Corregido: faltaba una coma aquí
            "Cantidad_de_compra", 
            "Cantidad_de_ventas",
            "Monto_de_ventas",
            "Fase_CAS",
            "Main_Group",
            "Grupo"
        ]
        
        # Verificar que todas las columnas existan en el DataFrame
        columnas_disponibles = [col for col in columnas_seleccionadas if col in df_compras.columns]
        if len(columnas_disponibles) < len(columnas_seleccionadas):
            columnas_faltantes = set(columnas_seleccionadas) - set(columnas_disponibles)
            print(f"Advertencia: Algunas columnas no existen en el DataFrame: {columnas_faltantes}")
            print(f"Columnas disponibles: {df_compras.columns.tolist()}")
        
        df_procesado = df_compras[columnas_disponibles].copy()
        
        # Convertir columnas numéricas
        columnas_numericas = [
            "Cantidad_de_stock",
            "Valor_de_stock_disponible",
            "Cantidad_de_compra",
            "Cantidad_de_ventas",
            "Monto_de_ventas"
        ]
        
        for columna in columnas_numericas:
            if columna in df_procesado.columns:
                df_procesado[columna] = pd.to_numeric(df_procesado[columna], errors='coerce')
                print(f"Columna '{columna}' convertida a tipo numérico")
        
        # Reemplazar valores NaN
        df_procesado = df_procesado.replace('nan', '')
        
        # Aplicar filtros
        if "Descripci_n" in df_procesado.columns:
            df_procesado = df_procesado[df_procesado["Descripci_n"] != ""]
        
        if "Cantidad_de_compra" in df_procesado.columns:
            df_procesado = df_procesado[df_procesado["Cantidad_de_compra"] != 0]
        
        if "Descripci_n" in df_procesado.columns:
            df_procesado = df_procesado[~df_procesado["Descripci_n"].str.contains("CONSUMO", na=False)]

        
        return df_procesado
    
    @classmethod
    def limpiar_df_para_bigquery_ventas(cls, df):
        """
        Realiza una limpieza completa del DataFrame para la tabla de ventas en BigQuery.
        
        Args:
            df (pandas.DataFrame): DataFrame a limpiar.
            
        Returns:
            pandas.DataFrame: DataFrame limpio y procesado para análisis de ventas.
        """
        # Normalizar nombres de columnas
        df_limpio = cls.normalizar_nombres_columnas(df)
        
        # Convertir a string para evitar problemas de compatibilidad
        df_limpio = df_limpio.astype(str)
        
        # Eliminar duplicados
        df_limpio = df_limpio.drop_duplicates()
        
        # Selección de columnas específicas
        listado_columnas = [
            "N_mero_de_producto_del_vendedor",
            "N_mero_de_producto_Makro",
            "Descripci_n",
            "Ubicaci_n",
            "Cantidad_de_stock",
            "Valor_de_stock_disponible",
            "Cantidad_de_compra",
            "Cantidad_de_ventas",
            "Monto_de_ventas"
        ]
        
        # Verificar columnas disponibles
        columnas_disponibles = [col for col in listado_columnas if col in df_limpio.columns]
        
        # Seleccionar solo las columnas requeridas que existen
        df_gold_ventas = df_limpio[columnas_disponibles]
        
        # Generar tabla final procesada
        df_makro = cls.procesar_ventas(df_gold_ventas)
        
        return df_makro
        
    @classmethod
    def limpiar_df_para_bigquery_inventory(cls, df):
        """
        Realiza una limpieza completa del DataFrame para la tabla de inventario en BigQuery.
        
        Args:
            df (pandas.DataFrame): DataFrame a limpiar.
            
        Returns:
            pandas.DataFrame: DataFrame limpio y procesado para análisis de inventario.
        """
        # Normalizar nombres de columnas
        df_limpio = cls.normalizar_nombres_columnas(df)
        
        # Convertir a string para evitar problemas de compatibilidad
        df_limpio = df_limpio.astype(str)
        
        # Eliminar duplicados
        df_limpio = df_limpio.drop_duplicates()
        
        # Selección de columnas específicas para inventario
        listado_columnas = [
            "N_mero_de_producto_del_vendedor",
            "N_mero_de_producto_Makro",
            "Descripci_n",
            "Ubicaci_n",
            "Cantidad_de_stock",
            "Valor_de_stock_disponible",
            "Cantidad_de_compra",
            "Cantidad_de_ventas",
            "Monto_de_ventas",
            "Fase_CAS",
            "Main_Group",
            "Grupo"
        ]
        
        # Verificar columnas disponibles
        columnas_disponibles = [col for col in listado_columnas if col in df_limpio.columns]
        
        if len(columnas_disponibles) < len(listado_columnas):
            print(f"Advertencia: Algunas columnas para inventario no están disponibles")
            print(f"Columnas faltantes: {set(listado_columnas) - set(columnas_disponibles)}")
        
        # Seleccionar solo las columnas requeridas que existen
        df_gold_inventario = df_limpio[columnas_disponibles]
        
        # Generar tabla final procesada para inventario
        df_makro_inventario = cls.procesar_compras(df_gold_inventario)
        
        return df_makro_inventario
        
    @classmethod
    def limpiar_df_para_bigquery(cls, df):
        """
        Realiza una limpieza completa del DataFrame para BigQuery (método mantenido por compatibilidad).
        
        Args:
            df (pandas.DataFrame): DataFrame a limpiar.
            
        Returns:
            pandas.DataFrame: DataFrame limpio y procesado para análisis de ventas.
        """
        return cls.limpiar_df_para_bigquery_ventas(df)


class BigQueryUploader:
    """
    Clase para subir DataFrames a Google BigQuery.
    
    Esta clase proporciona métodos para conectarse a BigQuery
    y subir datos desde un DataFrame de pandas.
    """
    
    def __init__(self, project_id, dataset_id, credentials_path):
        """
        Inicializa el cargador de BigQuery.
        
        Args:
            project_id (str): ID del proyecto de Google Cloud.
            dataset_id (str): ID del dataset de BigQuery.
            credentials_path (str): Ruta al archivo de credenciales.
        """
        self.project_id = project_id
        self.dataset_id = dataset_id
        self.credentials_path = credentials_path
        self.credentials = self._cargar_credenciales()
        
    def _cargar_credenciales(self):
        """
        Carga las credenciales de servicio de Google Cloud.
        
        Returns:
            Credentials: Objeto de credenciales para autenticación.
        """
        return service_account.Credentials.from_service_account_file(
            self.credentials_path,
            scopes=["https://www.googleapis.com/auth/cloud-platform"]
        )
    
    def subir_dataframe(self, df, table_id, if_exists='replace'):
        """
        Sube un DataFrame a una tabla de BigQuery.
        
        Args:
            df (pandas.DataFrame): DataFrame a subir.
            table_id (str): ID de la tabla de BigQuery.
            if_exists (str): Acción a tomar si la tabla ya existe 
                             ('fail', 'replace', o 'append').
                             
        Returns:
            bool: True si la carga fue exitosa, False en caso contrario.
        """
        if df is None or df.empty:
            print("Error: No hay datos para subir a BigQuery")
            return False
        
        try:
            destination_table = f"{self.project_id}.{self.dataset_id}.{table_id}"
            
            df.to_gbq(
                destination_table=destination_table,
                project_id=self.project_id,
                if_exists=if_exists,
                credentials=self.credentials
            )
            
            print(f"DataFrame subido exitosamente a {destination_table}")
            print(f"Total de filas subidas: {len(df)}")
            return True
            
        except Exception as e:
            print(f"Error al subir datos a BigQuery: {str(e)}")
            return False


def main():
    """
    Función principal que orquesta el flujo de trabajo completo.
    """
    # Configuración
    directorio = "../../"
    patron = "*Stock and PO*"
    header = 16
    credentials_path = "credentials/croc-454221-e1a3c2e02181.json"
    project_id = "croc-454221"
    dataset_id = "db_sales"
    table_id_ventas = "tbl_gld_sales_report_b2b_makro"
    table_id_inventario = "tbl_gld_sales_report_b2b_makro_inventory"
    
    # Procesamiento de archivos Excel
    print("\n=== PROCESAMIENTO DE ARCHIVOS EXCEL ===")
    excel_processor = ExcelProcessor(directorio, patron, header)
    df_raw = excel_processor.procesar_archivos(mostrar_stats=True)
    
    if df_raw is None:
        print("No se encontraron datos para procesar. Finalizando programa.")
        return
    
    # Guardar archivo consolidado (opcional)
    excel_processor.guardar_excel(df_raw, "datos_consolidados_raw.xlsx")
    
    # Crear instancia para BigQuery
    uploader = BigQueryUploader(project_id, dataset_id, credentials_path)
    resultados_ok = []
    
    # === PROCESO PARA TABLA DE VENTAS ===
    print("\n=== LIMPIEZA Y TRANSFORMACIÓN DE DATOS PARA TABLA DE VENTAS ===")
    df_ventas = DataFrameCleaner.limpiar_df_para_bigquery_ventas(df_raw)
    print("Proceso realizado correctamente ")
    print(f"\n=== CARGA A BIGQUERY: TABLA {table_id_ventas} ===")
    resultado_ventas = uploader.subir_dataframe(df_ventas, table_id_ventas, if_exists='replace')
    resultados_ok.append(resultado_ventas)
    
    if resultado_ventas:
        print(f"Carga de datos de ventas completada: {len(df_ventas)} filas")
    else:
        print("Error en la carga de datos de ventas")
    

    # === PROCESO PARA TABLA DE INVENTARIO ===
    print("\n=== LIMPIEZA Y TRANSFORMACIÓN DE DATOS PARA TABLA DE INVENTARIO ===")
    df_inventario = DataFrameCleaner.limpiar_df_para_bigquery_inventory(df_raw)
    
    print(f"\n=== CARGA A BIGQUERY: TABLA {table_id_inventario} ===")
    resultado_inventario = uploader.subir_dataframe(df_inventario, table_id_inventario, if_exists='replace')
    resultados_ok.append(resultado_inventario)
    
    if resultado_inventario:
        print(f"Carga de datos de inventario completada: {len(df_inventario)} filas")
    else:
        print("Error en la carga de datos de inventario")
    
    # Resumen final
    if all(resultados_ok):
        print("\n=== PROCESO COMPLETADO EXITOSAMENTE ===")
        print(f"- Tabla de ventas: {len(df_ventas)} filas cargadas")
        print(f"- Tabla de inventario: {len(df_inventario)} filas cargadas")
    else:
        print("\n=== EL PROCESO FINALIZÓ CON ERRORES ===")
        if resultado_ventas:
            print("- Carga de tabla de ventas: OK")
        else:
            print("- Carga de tabla de ventas: FALLÓ")
            
        if resultado_inventario:
            print("- Carga de tabla de inventario: OK")
        else:
            print("- Carga de tabla de inventario: FALLÓ")


if __name__ == "__main__":
    main()


=== PROCESAMIENTO DE ARCHIVOS EXCEL ===
Se encontraron 0 archivos con el patrón '*Stock and PO*'
No se encontraron datos válidos para concatenar.
No se encontraron datos para procesar. Finalizando programa.
